# AI 이메일 자동 발송 시스템

이 노트북에서는 Ollama LLM과 SMTP를 활용하여 자연어 입력으로 이메일을 자동 생성하고 발송하는 시스템을 구현합니다.

## 1. 라이브러리 Import 및 설정

필요한 라이브러리를 import하고 모델 및 SMTP 서버 설정을 정의합니다.

- **ollama**: 로컬 LLM 모델과 통신
- **smtplib**: 이메일 발송 기능
- **email.mime**: 이메일 메시지 구성

In [1]:
import ollama
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

## 2. 전역 설정값 정의

LLM 모델명과 Gmail SMTP 서버 설정을 정의합니다.

**주의**: 실제 사용 시 Gmail 앱 비밀번호를 발급받아 사용해야 합니다.

In [2]:
# 설정값
MODEL_NAME = "llama3.1"
SMTP_CONFIG = {
    'smtp_server': 'smtp.gmail.com',
    'smtp_port': 587,
    'email': '', # 자신의 이메일 입력
    'password': '' #자신의 앱 비밀번호 입력
}

## 3. 이메일 주소와 목적 추출 함수

자연어 입력에서 LLM을 사용하여 이메일 주소와 발송 목적을 지능적으로 추출합니다.

**핵심 기능**:
- 한국어 자연어 처리
- 불완전한 이메일 주소 자동 보완
- 컨텍스트 기반 목적 추출

In [3]:
def extract_email_and_purpose(user_input):
    """LLM을 사용해서 이메일 주소와 목적을 맥락적으로 추출"""
    try:
        prompt = f"""
        Extract email address and purpose from the following Korean user request:
        
        User request: "{user_input}"
        
        Please respond in this exact format:
        이메일: [extracted email address]
        목적: [email purpose/content in Korean]
        
        Rules:
        1. If email is incomplete (e.g., user@domain), automatically add .com
        2. Extract only the core purpose for the email content
        3. Remove unnecessary words like "메일 보내줘", "으로", "에게"
        4. Understand the context and extract accurately
        5. Always respond with "이메일:" and "목적:" labels in Korean
        """
        
        response = ollama.chat(
            model=MODEL_NAME,
            messages=[{'role': 'user', 'content': prompt}]
        )
        
        content = response['message']['content']
        print(f"LLM 응답: {content}")
        
        # 응답에서 이메일과 목적 추출
        email = None
        purpose = None
        
        for line in content.split('\n'):
            line = line.strip()
            if line.startswith("이메일:"):
                email = line.replace("이메일:", "").strip()
            elif line.startswith("목적:"):
                purpose = line.replace("목적:", "").strip()
        
        return email, purpose
        
    except Exception as e:
        print(f"LLM 추출 오류: {e}")
        return None, None

## 4. 이메일 내용 생성 함수

추출된 목적을 바탕으로 LLM이 적절한 이메일 제목과 내용을 자동 생성합니다.

**특징**:
- 간결하고 전문적인 한국어 이메일 작성
- 제목과 본문 분리 생성
- 최대 길이 제한으로 간결함 유지

In [4]:
def generate_email(purpose):
    """목적에 따라 이메일 제목과 내용 생성"""
    try:
        prompt = f"""
        Write a simple and brief professional email in Korean based on the following purpose:
        Purpose: {purpose}
        
        Please respond in this exact format:
        제목: [simple subject - maximum 10 words]
        내용: [brief and simple message in Korean - maximum 3 sentences]
        
        Requirements:
        - Write in Korean language
        - Keep it very simple and brief
        - No templates, placeholders, or formal structures
        - Just a natural, short message
        - Be direct and to the point
        """
        
        response = ollama.chat(
            model=MODEL_NAME,
            messages=[{'role': 'user', 'content': prompt}]
        )
        
        content = response['message']['content']
        print(f"이메일 생성 응답: {content}")
        
        # 제목과 내용 분리
        lines = content.split('\n')
        subject = ""
        body = ""
        
        current_section = None
        
        for line in lines:
            line = line.strip()
            if line.startswith("제목:"):
                subject = line.replace("제목:", "").strip()
                current_section = "subject"
            elif line.startswith("내용:"):
                body = line.replace("내용:", "").strip()
                current_section = "body"
            elif current_section == "body" and line:
                body += "\n" + line
        
        # 빈 값 체크
        if not subject:
            subject = "메일 발송"
        if not body:
            body = f"{purpose}에 대해 연락드립니다."
        
        return subject, body
        
    except Exception as e:
        print(f"이메일 생성 오류: {e}")
        return "이메일 제목", "이메일 내용을 생성할 수 없습니다."

## 5. 이메일 발송 함수

SMTP를 통해 실제 이메일을 발송하는 기능을 구현합니다.

**구현 내용**:
- Gmail SMTP 서버 연결
- TLS 보안 연결 설정  
- UTF-8 인코딩으로 한글 지원
- 에러 처리 및 연결 종료

In [5]:
def send_email(to_email, subject, body):
    """이메일 발송"""
    try:
        msg = MIMEMultipart()
        msg['From'] = SMTP_CONFIG['email']
        msg['To'] = to_email
        msg['Subject'] = subject
        
        msg.attach(MIMEText(body, 'plain', 'utf-8'))
        
        server = smtplib.SMTP(SMTP_CONFIG['smtp_server'], SMTP_CONFIG['smtp_port'])
        server.starttls()
        server.login(SMTP_CONFIG['email'], SMTP_CONFIG['password'])
        
        text = msg.as_string()
        server.sendmail(SMTP_CONFIG['email'], to_email, text)
        server.quit()
        
        print(f"이메일 발송 완료: {to_email}")
        return True
        
    except Exception as e:
        print(f"발송 실패: {e}")
        return False

## 6. 통합 실행 함수

모든 기능을 통합하여 사용자 입력 하나로 전체 프로세스를 실행합니다.

**전체 플로우**:
1. 사용자 입력 분석
2. 이메일 주소 및 목적 추출
3. 이메일 내용 생성
4. 실제 이메일 발송
5. 결과 피드백 제공

In [6]:
def send_simple_email(user_input):
    """사용자 입력 하나로 이메일 발송"""
    print("LLM이 요청을 분석 중...")
    
    # 1. 이메일 주소와 목적 추출
    email, purpose = extract_email_and_purpose(user_input)
    
    if not email:
        print("이메일 주소를 찾을 수 없습니다.")
        return False
        
    if not purpose:
        print("이메일 목적을 파악할 수 없습니다.")
        return False
    
    print(f"\n추출된 정보:")
    print(f"  받는 사람: {email}")
    print(f"  목적: {purpose}")
    
    # 2. 이메일 생성
    subject, body = generate_email(purpose)
    
    print(f"\n생성된 이메일:")
    print(f"  제목: {subject}")
    print(f"  내용: {body}")
    
    # 3. 이메일 발송
    return send_email(email, subject, body)

## 실습
실제 사용 예시로 자연어 명령을 통해 이메일을 발송합니다.

LLM이 아래 실습조교 이메일 주소로 **실습 진행 완료 여부** 를 내용을 담은 메일을 전송할 수 있도록 지시사항을 작성하세요

polpy29@gmail.com

힌트:"(메일내용구성)(메일주소)으로 메일을 보내줘

In [7]:
user_input = "고맙다고 해줘"
send_simple_email(user_input)

LLM이 요청을 분석 중...


KeyboardInterrupt: 